In [ ]:
!pip install --upgrade google-cloud-storage

In [ ]:
import os
from google.cloud import storage
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/angular-yeti-389702-d5b06204e451.json'

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .config("spark.jars", "gcs-connector-hadoop3-latest.jar") \
        .getOrCreate()

spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
# This is required if you are using service account and set true,
spark._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.enable', 'true')
spark._jsc.hadoopConfiguration().set('google.cloud.auth.service.account.json.keyfile', "angular-yeti-389702-d5b06204e451.json")


In [ ]:
# Read data from google cloud storage
def read_file_from_cloud():

  # ************* your code here **********************

  return df

In [ ]:
#Question 11
df =  read_file_from_cloud()
print(df.count())

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# Read data from google drive
def read_file_from_drive():
  # ************* your code here **********************
  return data

In [ ]:
#Question 11
df =  read_file_from_drive()
print(df.count())

506


In [ ]:
#Question 12
df =  read_file_from_drive()
result = df.select("crim","zn","indus","medv").orderBy("medv").toPandas()
print(round(result.iloc[0,-1],2))

5.0


In [ ]:
#Question 12
df =  read_file_from_cloud()
df.createOrReplaceTempView("df")
query = """
          SELECT crim, zn, indus, medv
          FROM df
          order by medv
        """
result = spark.sql(query).toPandas()
print(round(result.iloc[0,-1],2))

5.0


In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

def prepare_data_using_pyspark(df):

    # ************* your code here **********************


  return final_data




DataFrame[features: vector, medv: double]


In [ ]:
#Question 13
df =  read_file_from_drive() # or df =  read_file_from_cloud()
data = prepare_data_using_pyspark(df)
print(data)

DataFrame[features: vector, medv: double]


In [ ]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
def train(train_data):
  lr = LinearRegression(featuresCol="features", labelCol="medv", predictionCol="predicted_medv")

   # ************* your code here **********************

  return lr_model

In [ ]:
# Question 14
df =  read_file_from_drive() # or df =  read_file_from_cloud()
data = prepare_data_using_pyspark(df)
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
lr_model = train(train_data)
coefficients = lr_model.coefficients
intercept = lr_model.intercept

print("Coefficients: ", coefficients)
print("Intercept: {:.3f}".format(intercept))

Coefficients:  [-0.11362203729409111,0.048909186934055354,0.023795428986748923,2.801771998735071,-18.415424541191168,3.5158797633118337,0.005211682161471176,-1.4163830723540525,0.3317669315937271,-0.013607893704165114,-0.953414333840846,0.008602677392852453,-0.5195035312476768]
Intercept: 38.617


In [ ]:
#Question 15

from pyspark.ml.evaluation import RegressionEvaluator
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
lr_model = train(train_data)
predictions = lr_model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="medv", predictionCol="predicted_medv", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))

evaluator_r2 = RegressionEvaluator(labelCol="medv", predictionCol="predicted_medv", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R-squared (R2) on test data: {:.3f}".format(r2))

Root Mean Squared Error (RMSE) on test data: 4.672
R-squared (R2) on test data: 0.793


In [ ]:
#Question 16

assembler = VectorAssembler(
inputCols=["crim", "zn", "indus", "chas", "nox", "rm", "age", "dis", "rad", "tax", "ptratio", "b", "lstat"],
outputCol="features")
data = assembler.transform(df)

feature_importance = sorted(list(zip(data.columns[:-1], map(abs, coefficients))), key=lambda x: x[1], reverse=True)

print("The most important feature:", feature_importance[0][0])

The most important feature: nox
